# Web scraping with cbs_utils

In this notebook some small examples are given on how to use the web scraping utilities from cbs_utils

## Using the get_page_from_url function

The *get_page_from_url* function allow to obtain the contents of an url and store the results in cache. The next time you run the function again, the function is read from cache. Here, an small example is given. First start with import the required modules:

In [30]:
import logging
from pathlib import Path

from bs4 import BeautifulSoup

from cbs_utils.misc import (create_logger, merge_loggers)
from cbs_utils.regular_expressions import (KVK_REGEXP, ZIP_REGEXP, BTW_REGEXP)
from cbs_utils.web_scraping import (get_page_from_url, UrlSearchStrings)

Set up the logging module using the cbs_utils misc function *create_logger*

In [31]:
# set up logging
log_level = logging.DEBUG  # change to DEBUG for more info
log_format = logging.Formatter('%(levelname)8s --- %(message)s')
logger = create_logger(console_log_level=log_level, formatter=log_format)
merge_loggers(logger, "cbs_utils.web_scraping", logger_level_to_merge=logging.INFO)

<Logger cbs_utils.web_scraping (INFO)>

For this example a *tmp* directory is made in your working directory to store the cache. First make sure we clean this directory in case it still existed from the previous run

In [32]:
# create url name and clean previous cache file
cache_directory = Path("tmp")
clean_cache = True
if clean_cache:
    if cache_directory.exists():
        for item in cache_directory.iterdir():
            item.unlink()
        cache_directory.rmdir()
    else:
        logger.info(f"Cache directory {cache_directory} was already removed")


Now we can demonstrate the *get_page_from_url* function. Note that the *with Timer* construct is only added to be able to report the processing time

In [33]:
%%time
url = "https://www.example.com"
page = get_page_from_url(url, cache_directory=cache_directory)

CPU times: user 35 ms, sys: 3 ms, total: 38 ms
Wall time: 5.43 s


As you can see, it took about 5.5 s to get all the information from the internet. Because we have added a *cache_to_disk* iterator to the *get_page_from_url* function, a cache wil in the *tmp* directory was made:

In [34]:
for item in cache_directory.iterdir():
    logger.info(f"Cache file: {item}")

    INFO --- Cache file: tmp/get_page_from_url_https_www_example_com_.pkl


The contents of the url was stored in page and look like this:

In [35]:
soup = BeautifulSoup(page.text, 'lxml')
logger.info("\n{}\n".format(soup.body))

    INFO --- 
<body>
<div>
<h1>Example Domain</h1>
<p>This domain is established to be used for illustrative examples in documents. You may use this
    domain in examples without prior coordination or asking for permission.</p>
<p><a href="http://www.iana.org/domains/example">More information...</a></p>
</div>
</body>



We can run the same function again. Since we now have a cache file, it will be about 1000 x faster:

In [44]:
%%time
page2 = get_page_from_url(url, cache_directory=cache_directory)


CPU times: user 0 ns, sys: 1 ms, total: 1 ms
Wall time: 783 µs


Indeed the contecnt with exactely the same function statement runs in with about 1 ms. Now compare the results:

In [46]:
soup2 = BeautifulSoup(page2.text, 'lxml')
logger.info("Contents is equal: {}".format(soup.body == soup2.body))

    INFO --- Contents is equal: True


# Using the *UrlSearchStrings* class

The *UrlSearchString* class can be used to recursively crawl a web site and search for a list of regular expression we want to obtained from the web site. Again, the result is cached, so in case you want to run it again with different search string it will run significantly faster. 

Let first set up our first search session, trying to retrieve the postal code and kvk number from a web page

In [ ]:
%%time
# the regular expression are obtained from the cbs_utils.regular_expressions module
searches = dict(
    postcode=ZIP_REGEXP,
    kvknumber=KVK_REGEXP
)

url = "www.be-one.nl"
url_analyse = UrlSearchStrings(url, search_strings=searches, cache_directory=cache_directory,
                               store_page_to_cache=True)

    INFO --- Get (cached) page: https://www.be-one.nl/ with validate True (web_scraping.py:820)
    INFO --- Get (cached) page: https://www.be-one.nl/ with validate True
    INFO --- Get (cached) page: https://www.be-one.nl/ with validate True
